In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import time

In [ ]:
import torch
import numpy as np

#sys.path.insert(1, os.path.join("C:\\", "Users", "Soham", "Desktop", "CovNet", "source_codes"))
## modify as needed

import CovNetworks as CN
import Important_functions as Ifn
import Other_functions as Ofn

import current_setup as setup

### Update

method = input("Model [Shallow/Deepshared/Deep]: ")
if method.lower() == 'deepshared' or method.lower() == 'deep':
    depth = input("Depth of the network: ")
    depth = int(depth)
R = input("Number of components (R): ")
R = int(R)

act_fn = setup.act_fn
init = setup.init
#loss = 'COV'
loss_fn = Ifn.loss_COV

if method.lower()=='shallow':
    err_file = 'Err_'+method+'_'+str(R)+'.txt'
elif method.lower()=='deepshared' or method.lower() == 'deep':
    err_file = 'Err_'+method+'_'+str(depth)+'_'+str(R)+'.txt'

dirc = setup.directory
replicates = setup.replicates

In [ ]:
dirc = "C:\\Soham\\Git\\spectral-NN\\Data\\"
repl = 0

In [ ]:
    print('Example'+str(repl+1)+':')
    file = dirc+'locations'+str(repl+1)+'.dat'
    u = np.loadtxt(dirc+"locations.dat",dtype="float32")
    if len(u.shape)==1:
        D, d = len(u), 1
        u = u.reshape(D,1)
    else:
        D, d = u.shape
    u = torch.from_numpy(u)
    file = dirc+'Example'+str(repl+1)+'.dat'
    x = np.loadtxt(file,dtype='float32')
    N = x.shape[0]
    if x.shape[1] != D:
        exit('Data shape mismatch!! Aborting..')
    print('N='+str(N)+', D='+str(D)+', d='+str(d))

    x = torch.from_numpy(x)
    x = x - torch.mean(x,dim=0,keepdim=True)

for repl in replicates:
    print('Example'+str(repl+1)+':')
    file = dirc+'locations'+str(repl+1)+'.dat'
    u = np.loadtxt(file,dtype='float32')
    D, d = u.shape
    u = torch.from_numpy(u)
    file = dirc+'Example'+str(repl+1)+'.dat'
    x = np.loadtxt(file,dtype='float32')
    N = x.shape[0]
    if x.shape[1] != D:
        exit('Data shape mismatch!! Aborting..')
    print('N='+str(N)+', D='+str(D)+', d='+str(d))
    
    x = torch.from_numpy(x)
    #x = x - torch.mean(x,dim=0,keepdim=True)

    if method.lower()=='shallow':
        epochs = setup.epochs_shallow
        burn_in = setup.burn_in_shallow
        interval = setup.interval_shallow
        model = CN.CovNetShallow(d,N,R,act_fn,init)
        checkpoint_file = 'Best_'+method+'_'+str(R)+'.pt'
    elif method.lower()=='deepshared':
        epochs = setup.epochs_deepshared
        burn_in = setup.burn_in_deepshared
        interval = setup.interval_deepshared
        model = CN.CovNetDeepShared(d,N,R,depth,act_fn,init)
        checkpoint_file = 'Best_'+method+'_'+str(depth)+'_'+str(R)+'.pt'
    elif method.lower()=='deep':
        epochs = setup.epochs_deep
        burn_in = setup.burn_in_deep
        interval = setup.interval_deep
        n_nodes = R
        model = CN.CovNetDeep(d,N,R,depth,n_nodes,act_fn,init)
        checkpoint_file = 'Best_'+method+'_'+str(depth)+'_'+str(R)+'.pt'
        
    optimizer = setup.optimizer(model.params,lr=setup.lr)
    split = setup.split
    
    print(time.ctime())
    l_tr, l_va, epoch = Ifn.cnet_optim_best(x,u,model,loss_fn,optimizer,split,epochs,burn_in,interval,checkpoint_file)
    del x,u
    
    file = dirc+'True_locations'+str(repl+1)+'.dat'
    loc = np.loadtxt(file,dtype='float32')
    u = torch.from_numpy(loc[:,:d])
    v = torch.from_numpy(loc[:,d:])
    del loc
    cov_file = dirc+'True_cov'+str(repl+1)+'.dat'
    err = Ofn.cnet_error_MC(model,u,v,cov_file)
    
    f_Err = open(err_file,'a')
    f_Err.write('Example{}:\n' .format(repl+1))
    f_Err.write('Error = {:.10f}\n' .format(err))
    f_Err.write('Number of epochs = {}\n' .format(epoch))
    f_Err.close()
    print('Error = {:.6f}'.format(err))
    print('Number of epochs = {}' .format(epoch))
    print(time.ctime())
    print('\n')
    os.remove(checkpoint_file)

In [ ]:
class spectralNNShallow(torch.nn.Module):
    def __init__(self,N,d,M,L,act_fn=torch.nn.Sigmoid(),init=torch.nn.init.xavier_normal_):
        super(spectralNNShallow, self).__init__()
        self.act_fn = act_fn
        self.init = init
        self.par = torch.zeros([M,L,d],dtype=torch.float32,requires_grad=True)
        self.init(self.par)
        self.xi = torch.ones([N,M,L],dtype=torch.float32,requires_grad=True)
        self.init(self.xi)
        self.params = list([self.par, self.xi])
        ### add bias term

    def first_step(self, u):
        return self.act_fn(torch.einsum("ijk,lk -> ijl", self.par, u))

    def forward(self, u):
        return torch.einsum("ijk,jkl -> il", self.xi, self.first_step(u))

In [ ]:
M = 10
L = 10
#N = 200
#d = 1
#K = 100
#D = K**d

In [ ]:
#x = np.array(np.random.randn(N,D),dtype="float32")
#u = np.array(np.arange(1,K+1)/(K+1),dtype="float32").reshape(-1,1)
#print(x.shape)
#print(u.shape)
#x = torch.from_numpy(x)
#u = torch.from_numpy(u)

In [ ]:
model = spectralNNShallow(N,d,M,L)
#print(model.params)
#x_hat = model(u)
#loss = torch.norm(x-x_hat)
#print(loss.item())

In [ ]:
optimizer = torch.optim.Adam(model.params,lr=0.01)

In [ ]:
for i in range(1000):
    loss = torch.norm(x-model(u))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss.item())

In [ ]:
print(model.params)

In [ ]:
par = torch.zeros([M,L,d],dtype=torch.float32,requires_grad=True)
xi = torch.ones([N,M,L],dtype=torch.float32,requires_grad=True)

In [ ]:
torch.nn.init.xavier_normal_(par)
print(par)